In [41]:
import pandas as pd
import numpy as np
import dotenv
from incawrapper import dataschemas

In [2]:
# import environment variables
INCA_base_directory = dotenv.get_key(dotenv.find_dotenv(), "INCA_base_directory")

In [3]:
xl = pd.ExcelFile("./Literature data/Arabidopsis thaliana Allen 2014/pnas.1319485111.sd01.xls")

In [4]:
xl.sheet_names

['Overview',
 'MRM Parameters',
 'GCMS fragments',
 'LCMSMS transitions',
 'UDGP and ADPG Transitions',
 'Labeled UDPG and ADPG',
 'Metabolite recovery',
 'PGA',
 'DHAP',
 'S7P',
 'R5P',
 'P5P',
 'RUBP',
 'G6P',
 'F6P',
 'G1P',
 'FBP',
 '2PG',
 'Glycerate',
 'PEP',
 'ADPG',
 'UDPG',
 'Malate',
 'Succinate',
 'Fumarate',
 'Aconitate',
 'a-Ketoglutarate',
 'Citrate',
 'Ser',
 'Gly',
 'Asp',
 'Ala',
 'Pro',
 'Thr',
 'Asn',
 'Glu',
 'Gln',
 't=60',
 'Abbreviations',
 'Reactions',
 'Net fluxes',
 'Exchange fluxes',
 'Subcellular fluxes',
 'Dilution Parameters',
 'Pool Sizes']

In the data they authors describe a meta reaction called PhleomOut. This reaction as variable coefficients. In the following we define dictionary which describes which coefficient to use.

In [5]:
output_sink_ratios = {
    "α": "1", 
    "β":"1", 
    "γ":"1",
    "δ":"1",
    "ε":"1"
}

Now we are ready to read in the reactions and wrangle them into the correct format for the INCAWrapper.

In [6]:
reactions = (xl
    .parse('Reactions', skiprows=4, skipfooter=3)
    .rename(columns={
        "Calvin Cycle": "rxn_id", 
        "Unnamed: 1": "Substrate",
        "Unnamed: 2": "Arrow", 
        "Unnamed: 3": "Product"
    })
    .dropna(subset=["Arrow"]) # Removes section headers within the table
    .assign(
        Arrow= lambda x: x.Arrow.replace({"→":"->", "↔":"<->"})
    )
    .assign(
        rxn_eqn = lambda x: x.Substrate + " " + x.Arrow + " " + x.Product,
    )
    .drop(columns=["Substrate", "Arrow", "Product"])
)

# modify PhloemOut output sink reaction
reactions.loc[reactions.rxn_id == "PhloemOut", "rxn_eqn"] = (reactions
    .loc[reactions.rxn_id == "PhloemOut", "rxn_eqn"]
    .replace(
        output_sink_ratios, 
        regex=True
    )
    .replace(
        {"\(":"", "\)":""},
        regex=True
    )
)

dataschemas.ReactionsSchema.validate(reactions)

,rxn_id,rxn_eqn
0,RUBISCO_CO2,RUBP.p (abcde) + CO2 (f) -> 3PGA.p (cde) + 3P...
1,ALD,TP.p (abc) + E4P.p (defg) -> SBP (cbadefg)
2,SBPase,SBP (abcdefg) -> S7P.p (abcdefg)
3,TK1,TP.p (cde) + EC2 (ab) <-> X5P.p (abcde)
4,TK2,S7P.p (abcdefg) <-> R5P.p (cdefg) + EC2 (ab)
5,TK3,F6P.p (abcdef) <-> E4P.p (cdef) + EC2 (ab)
6,PPE,X5P.p (abcde) <-> RU5P.p (abcde)
7,PPI,R5P.p (abcde) <-> RU5P.p (abcde)
8,PRK,RU5P.p (abcde) -> RUBP.p (abcde)
10,RUBISCO_O2,RUBP.p (abcde) -> 3PGA.p (cde) + 2PG.p (ba)


In [7]:
mdv_sheet = (xl.parse("Asp"))
mdv_sheet


,Mass isotopomer measured abundances.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Compound,Labeling time,NaN,LL experiment,NaN,NaN,NaN,HL-ACC experiment,NaN,NaN
2,ASP 316,(min),Isotopomer,Rep. 1,Rep. 2,Rep. 3,Rep. 1,Rep. 2,Rep. 3,Rep. 4
3,NaN,0,[M]+,5588302,5229444.8,4980139.2,5616415.7,5705676,424875.7,509742.2
4,NaN,NaN,[M+1]+,1562679.1,1421519.9,1358543.2,1657950.5,1602144.6,115772.6,140531.7
...,...,...,...,...,...,...,...,...,...,...
151,NaN,15,[M]+,5779674.7,6033424.2,6160837.9,4713341.8,4198310.2,X,267147.9
152,NaN,NaN,[M+1]+,2329567,2533550.2,2638299.8,2050628,1879604.5,X,106610.5
153,NaN,NaN,[M+2]+,1323153.2,1457488.1,1571362.2,1249858.6,1063154.6,X,55248.4
154,NaN,NaN,[M+3]+,759208.9,741771.3,851424.6,892230.3,707085.4,X,23129.3


In [8]:
from typing import List
def extract_mdv_table(sheet: pd.DataFrame)->List[pd.DataFrame]:
    """One sheet can contain multiple MDV tables. This function extracts 
    them all and returns a list of DataFrames.
    """
    tables = []
    table = []
    for i, row in sheet.iterrows():
        if row.isna().all():
            if len(table) > 0:
                tables.append(pd.DataFrame(table))
                table = []
        else:
            table.append(row)
    if len(table) > 0:
        tables.append(pd.DataFrame(table))
    return tables

mdv_tables = extract_mdv_table(mdv_sheet)
mdv_table = mdv_tables[0]
mdv_table.head()


,Mass isotopomer measured abundances.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
1,Compound,Labeling time,NaN,LL experiment,NaN,NaN,NaN,HL-ACC experiment,NaN,NaN
2,ASP 316,(min),Isotopomer,Rep. 1,Rep. 2,Rep. 3,Rep. 1,Rep. 2,Rep. 3,Rep. 4
3,NaN,0,[M]+,5588302,5229444.8,4980139.2,5616415.7,5705676,424875.7,509742.2
4,NaN,NaN,[M+1]+,1562679.1,1421519.9,1358543.2,1657950.5,1602144.6,115772.6,140531.7
5,NaN,NaN,[M+2]+,595215.7,539683.7,518377.3,601837.8,639711.2,45061.2,54467.2


In [53]:
def convert_mass_isotope_to_int(mass_isotope: str)->int:
    """Converts mass_isotope str to an interger. Accepts strings like
    [M]+, [M+1]+ ...
    """
    if mass_isotope == "[M]+":
        return 0
    else:
        mass_isotope_int = mass_isotope.replace("[M+", "").replace("]+", "")
        return int(mass_isotope_int)


def tidy_mdv_table(mdv_table: pd.DataFrame, relative_error: float)->pd.DataFrame:
    ms_id = mdv_table.iloc[1,0]

    # manually preparing header 
    mdv_table.iloc[0,3:6] = 'LL experiment'
    mdv_table.iloc[0,6:10] = 'HL-ACC experiment'
    new_header = (mdv_table
        .iloc[:2]
        .fillna('')
        .apply(lambda row: "_".join(row)) #grab the first row for the header
        .reset_index(drop=True)
    )
    mdv_table.columns = new_header
    
    # remove header rows
    mdv_table = mdv_table.iloc[2:]

    tidy = (mdv_table
        .rename(
            columns={
                f"Compound_{ms_id}": "ms_id",
                "Labeling time_ (min)": "time",
                "_Isotopomer": "mass_isotope"
            }
        )
        .assign(ms_id = ms_id)
        .assign(time = lambda df: df.time.fillna(method="ffill"))
        .pipe(
            pd.wide_to_long,
            stubnames=["LL experiment", "HL-ACC experiment"],
            i=["ms_id", "time", "mass_isotope"],
            j="replicate",
            sep="_",
            suffix=r'.+'
        )
        .melt(ignore_index=False, var_name="experiment_id", value_name="intensity")
        .replace("X", np.nan)
        .reset_index()
        .assign(
            mass_isotope = lambda df: df.mass_isotope.apply(convert_mass_isotope_to_int),
        )
        .assign(
            metabolite_id = lambda df: df.ms_id.str.split(" ").str[0] # extract metabolite_id from ms_id
        )
        .assign(
            intensity_std_error = lambda df: df.intensity * relative_error # adding uncertainty 
        )
    )
    
    return tidy
tidy_mdv_table(mdv_table, 0.05)

,ms_id,time,mass_isotope,replicate,experiment_id,intensity,metabolite_id,intensity_std_error
0,ASP 316,0.0,0,Rep. 1,LL experiment,5588302.0,ASP,279415.100
1,ASP 316,0.0,0,Rep. 2,LL experiment,5229444.8,ASP,261472.240
2,ASP 316,0.0,0,Rep. 3,LL experiment,4980139.2,ASP,249006.960
3,ASP 316,0.0,0,Rep. 4,LL experiment,NaN,ASP,NaN
4,ASP 316,0.0,1,Rep. 1,LL experiment,1562679.1,ASP,78133.955
...,...,...,...,...,...,...,...,...
347,ASP 316,15.0,2,Rep. 4,HL-ACC experiment,41928.2,ASP,2096.410
348,ASP 316,15.0,3,Rep. 1,HL-ACC experiment,1144723.9,ASP,57236.195
349,ASP 316,15.0,3,Rep. 2,HL-ACC experiment,912355.2,ASP,45617.760
350,ASP 316,15.0,3,Rep. 3,HL-ACC experiment,NaN,ASP,NaN
